In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
import nltk # .word_tokenize() Tokenization

In [4]:
from nltk.corpus import stopwords # stopword
from nltk.stem.snowball import SnowballStemmer #Stemming
from nltk.stem import WordNetLemmatizer #lemmatizing
from sklearn.feature_extraction.text import CountVectorizer #count vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #TFIDF vectorizer

In [5]:
df =  pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df

Id                                            Comment      Topic
0      0x840  A few things. You might have negative- frequen...    Biology
1      0xbf0  Is it so hard to believe that there exist part...    Physics
2     0x1dfc                                     There are bees    Biology
3      0xc7e  I'm a medication technician. And that's alot o...    Biology
4      0xbba                     Cesium is such a pretty metal.  Chemistry
...      ...                                                ...        ...
8690  0x1e02  I make similar observations over the last week...    Biology
8691   0xc8d                                    You would know.    Biology
8692   0x723            Also use the correct number of sig figs  Chemistry
8693   0x667  What about the ethical delimmas,  groundbreaki...    Biology
8694  0x1476                          I would like to know too.    Biology

[8695 rows x 3 columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8695 entries, 0 to 8694
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       8695 non-null   object
 1   Comment  8695 non-null   object
 2   Topic    8695 non-null   object
dtypes: object(3)
memory usage: 203.9+ KB


## Data Cleaning

In [7]:
df.duplicated().sum()

0

`id is of no use, so we are dropping off`

In [8]:
df.drop('Id', axis=1, inplace=True)

## Data Preprocessing

In [9]:
label_encoder = LabelEncoder()

#### Label Encoding

In [10]:
label_encoder.fit(df.Topic)
df['Topic_Code'] = label_encoder.transform(df.Topic)
df_test['Topic_Code'] = label_encoder.transform(df_test.Topic)

In [11]:
df

Comment      Topic  Topic_Code
0     A few things. You might have negative- frequen...    Biology           0
1     Is it so hard to believe that there exist part...    Physics           2
2                                        There are bees    Biology           0
3     I'm a medication technician. And that's alot o...    Biology           0
4                        Cesium is such a pretty metal.  Chemistry           1
...                                                 ...        ...         ...
8690  I make similar observations over the last week...    Biology           0
8691                                    You would know.    Biology           0
8692            Also use the correct number of sig figs  Chemistry           1
8693  What about the ethical delimmas,  groundbreaki...    Biology           0
8694                          I would like to know too.    Biology           0

[8695 rows x 3 columns]

#### Tokenization, Stop word removal, Stemming, Lemmitizing

In [12]:
stop_words = set(stopwords.words('english')) # English stopwords
dict_words = set(nltk.corpus.words.words()) # English dictionary words
def text_clean(text):
    temp = []
    words = nltk.word_tokenize(text) # tokenization
#     setm_words = SnowballStemmer() # stemmer
    lemmatizer = WordNetLemmatizer() # Lemmatizer
    for word in words:
        word = word.lower()
        if word in stop_words or not(word.isalpha()) or not(word in dict_words): #remove stop words and alpha numeric
            pass
        else:
            temp.append(lemmatizer.lemmatize(word))
    return ' '.join(temp)
            
    

In [13]:
#Function Test
text_clean('The company has been hit with many challenges from recalls to price cuts $150000 and leadership concerns around CEO Elon Musk.')

'company hit many price leadership around musk'

In [14]:
%%time
df['Cleaned_Comments'] = df.Comment.apply(text_clean)

Wall time: 2.19 s


In [15]:
%%time
df_test['Cleaned_Comments'] = df_test.Comment.apply(text_clean)

Wall time: 967 ms


In [16]:
df

Comment      Topic  \
0     A few things. You might have negative- frequen...    Biology   
1     Is it so hard to believe that there exist part...    Physics   
2                                        There are bees    Biology   
3     I'm a medication technician. And that's alot o...    Biology   
4                        Cesium is such a pretty metal.  Chemistry   
...                                                 ...        ...   
8690  I make similar observations over the last week...    Biology   
8691                                    You would know.    Biology   
8692            Also use the correct number of sig figs  Chemistry   
8693  What about the ethical delimmas,  groundbreaki...    Biology   
8694                          I would like to know too.    Biology   

      Topic_Code                                   Cleaned_Comments  
0              0  might frequency dependent selection going leas...  
1              2  hard believe exist ca detect anything mean loo...  
2              0                                                bee  
3              0  medication technician liver probably wo die im...  
4              1                                cesium pretty metal  
...          ...                                                ...  
8690           0  make similar last southern seem crave energy s...  
8691           0                                         would know  
8692           1                        also use correct number sig  
8693           0  ethical technology future research field genet...  
8694           0                                    would like know  

[8695 rows x 4 columns]

#### Vectorizer

`COUNT VECTORIZER` : I have used TFIDF vectorizer (this is just for exploring)

In [17]:
count_vectorizer = CountVectorizer() # count vectorizer

In [18]:
count_vectorizer.fit(df.Cleaned_Comments)

CountVectorizer()

In [19]:
vector_count = count_vectorizer.transform((df.Cleaned_Comments))

In [20]:
temp_df_countvector = pd.DataFrame(vector_count.toarray(), columns = count_vectorizer.get_feature_names_out())

In [21]:
temp_df_countvector

abandon  abbreviation  abdomen  abdominal  abduction  abhor  ability  \
0           0             0        0          0          0      0        0   
1           0             0        0          0          0      0        0   
2           0             0        0          0          0      0        0   
3           0             0        0          0          0      0        0   
4           0             0        0          0          0      0        0   
...       ...           ...      ...        ...        ...    ...      ...   
8690        0             0        0          0          0      0        0   
8691        0             0        0          0          0      0        0   
8692        0             0        0          0          0      0        0   
8693        0             0        0          0          0      0        0   
8694        0             0        0          0          0      0        0   

      abiotic  abject  able  ...  zero  zig  zinc  zink  zip  zipper  zone  \
0           0       0     0  ...     0    0     0     0    0       0     0   
1           0       0     0  ...     0    0     0     0    0       0     0   
2           0       0     0  ...     0    0     0     0    0       0     0   
3           0       0     0  ...     0    0     0     0    0       0     0   
4           0       0     0  ...     0    0     0     0    0       0     0   
...       ...     ...   ...  ...   ...  ...   ...   ...  ...     ...   ...   
8690        0       0     0  ...     0    0     0     0    0       0     0   
8691        0       0     0  ...     0    0     0     0    0       0     0   
8692        0       0     0  ...     0    0     0     0    0       0     0   
8693        0       0     0  ...     0    0     0     0    0       0     0   
8694        0       0     0  ...     0    0     0     0    0       0     0   

      zoom  zwitterion  zygomatic  
0        0           0          0  
1        0           0          0  
2        0           0          0  
3        0           0          0  
4        0           0          0  
...    ...         ...        ...  
8690     0           0          0  
8691     0           0          0  
8692     0           0          0  
8693     0           0          0  
8694     0           0          0  

[8695 rows x 8677 columns]

`TFIDF VECTORIZER`

In [22]:
tfidf_vectorizer = TfidfVectorizer() # TFIDF vectorizer

In [23]:
tfidf_vectorizer.fit(df.Cleaned_Comments)

TfidfVectorizer()

In [24]:
vector_tfidf = tfidf_vectorizer.transform(df.Cleaned_Comments)

In [25]:
temp_df_tfidf = pd.DataFrame(vector_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names_out())

In [26]:
temp_df_tfidf

abandon  abbreviation  abdomen  abdominal  abduction  abhor  ability  \
0         0.0           0.0      0.0        0.0        0.0    0.0      0.0   
1         0.0           0.0      0.0        0.0        0.0    0.0      0.0   
2         0.0           0.0      0.0        0.0        0.0    0.0      0.0   
3         0.0           0.0      0.0        0.0        0.0    0.0      0.0   
4         0.0           0.0      0.0        0.0        0.0    0.0      0.0   
...       ...           ...      ...        ...        ...    ...      ...   
8690      0.0           0.0      0.0        0.0        0.0    0.0      0.0   
8691      0.0           0.0      0.0        0.0        0.0    0.0      0.0   
8692      0.0           0.0      0.0        0.0        0.0    0.0      0.0   
8693      0.0           0.0      0.0        0.0        0.0    0.0      0.0   
8694      0.0           0.0      0.0        0.0        0.0    0.0      0.0   

      abiotic  abject  able  ...  zero  zig  zinc  zink  zip  zipper  zone  \
0         0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
1         0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
2         0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
3         0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
4         0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
...       ...     ...   ...  ...   ...  ...   ...   ...  ...     ...   ...   
8690      0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
8691      0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
8692      0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
8693      0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   
8694      0.0     0.0   0.0  ...   0.0  0.0   0.0   0.0  0.0     0.0   0.0   

      zoom  zwitterion  zygomatic  
0      0.0         0.0        0.0  
1      0.0         0.0        0.0  
2      0.0         0.0        0.0  
3      0.0         0.0        0.0  
4      0.0         0.0        0.0  
...    ...         ...        ...  
8690   0.0         0.0        0.0  
8691   0.0         0.0        0.0  
8692   0.0         0.0        0.0  
8693   0.0         0.0        0.0  
8694   0.0         0.0        0.0  

[8695 rows x 8677 columns]

## Model Fitting

In [27]:
from sklearn.svm import SVC

In [28]:
model_scv = SVC()

In [29]:
%%time
model_scv.fit(vector_tfidf.toarray(), df.Topic_Code)

Wall time: 7min 36s


SVC()

In [30]:
%%time
# Prediction on Train data
prediction = model_scv.predict(vector_tfidf.toarray())

Wall time: 13min


In [31]:
%%time
#prediction for  Test data
prediction_test = model_scv.predict(tfidf_vectorizer.transform(df_test.Cleaned_Comments).toarray())

Wall time: 2min 23s


In [32]:
from sklearn.metrics import accuracy_score

#### Accuracy Score

In [33]:
print('Train Accuracy (SVC model) :', accuracy_score(df.Topic_Code, prediction))
print('Test Accuracy (SVC model) :', accuracy_score(df_test.Topic_Code, prediction_test))

Train Accuracy (SVC model) : 0.9254744105807936
Test Accuracy (SVC model) : 0.7919293820933165


In [34]:
from sklearn.metrics import confusion_matrix, classification_report

In [35]:
print(confusion_matrix(df_test.Topic_Code, prediction_test))

[[516  69  29]
 [ 77 406  23]
 [ 71  61 334]]


In [36]:
print(classification_report(df_test.Topic_Code, prediction_test))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81       614
           1       0.76      0.80      0.78       506
           2       0.87      0.72      0.78       466

    accuracy                           0.79      1586
   macro avg       0.80      0.79      0.79      1586
weighted avg       0.80      0.79      0.79      1586



#### Model Testing

In [37]:
def test_model_svc(x):
    data = text_clean(x) # tokenize + stop words removal + lemmatize
    data = tfidf_vectorizer.transform([data]) # converting to vectors
    data = model_scv.predict(data.toarray()) # prediction
    data = label_encoder.inverse_transform(data) #inverse encoding of prediction
    return data[0]

In [38]:
test_model_svc('hello cells this is ameoba')

'Biology'

In [39]:
test_model_svc('force of gravitation is the highest')

'Physics'

In [40]:
test_model_svc('water contains oxygen and hydrogen in 2:1')

'Chemistry'

#### Random Forest 

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
model_randomforest = RandomForestClassifier()

In [43]:
%%time
model_randomforest.fit(vector_tfidf.toarray(), df.Topic_Code)

Wall time: 4min 8s


RandomForestClassifier()

In [44]:
%%time
## prediction for train data
prediction_train_rf = model_randomforest.predict(vector_tfidf.toarray())

Wall time: 3.2 s


In [45]:
%%time
## prediction for test data
prediction_test_rf = model_randomforest.predict(tfidf_vectorizer.transform(df_test.Cleaned_Comments).toarray())

Wall time: 510 ms


In [46]:
print('Train Accuracy (Random Forest model) :', accuracy_score(df.Topic_Code, prediction_train_rf))
print('Test Accuracy (Random Forest model) :', accuracy_score(df_test.Topic_Code, prediction_test_rf))

Train Accuracy (Random Forest model) : 0.9529614721104083
Test Accuracy (Random Forest model) : 0.7503152585119798


#### Hyperparameter Tunning (Random Forest)

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
grid_params = {
              'n_estimators': [50, 100, 200],
              'max_depth': [None, 10, 20],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]
              }

In [49]:
grid_search = GridSearchCV(model_randomforest,grid_params, cv=5, scoring='accuracy', verbose=1, n_jobs=-1 )

In [50]:
%%time
grid_search.fit(vector_tfidf.toarray(), df.Topic_Code)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Wall time: 3h 53min 27s


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [51]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best Score: 0.6431282346175963


In [52]:
%%time
test_score = grid_search.score(tfidf_vectorizer.transform(df_test.Cleaned_Comments).toarray(), df_test.Topic_Code)
print("Test Score:", test_score)

Test Score: 0.7484237074401009
Wall time: 668 ms


#### Model Testing

In [56]:
model_randomforest_best = RandomForestClassifier(max_depth=None,  min_samples_leaf=1,min_samples_split=10,n_estimators=200)

In [57]:
%%time
model_randomforest_best.fit(vector_tfidf.toarray(), df.Topic_Code)

Wall time: 8min 8s


RandomForestClassifier(min_samples_split=10, n_estimators=200)

In [59]:
%%time
## prediction for train data
prediction_train_rf_best = model_randomforest_best.predict(vector_tfidf.toarray())
## prediction for test data
prediction_test_rf_best = model_randomforest_best.predict(tfidf_vectorizer.transform(df_test.Cleaned_Comments).toarray())

Wall time: 6.72 s


In [60]:
print('Train Accuracy (Random Forest model) :', accuracy_score(df.Topic_Code, prediction_train_rf_best))
print('Test Accuracy (Random Forest model) :', accuracy_score(df_test.Topic_Code, prediction_test_rf_best))

Train Accuracy (Random Forest model) : 0.9412305922944221
Test Accuracy (Random Forest model) : 0.7522068095838588


In [61]:
print(classification_report(df_test.Topic_Code, prediction_test_rf_best))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76       614
           1       0.73      0.71      0.72       506
           2       0.77      0.76      0.77       466

    accuracy                           0.75      1586
   macro avg       0.75      0.75      0.75      1586
weighted avg       0.75      0.75      0.75      1586



In [62]:
def test_model_rf(x):
    data = text_clean(x) # tokenize + stop words removal + lemmatize
    data = tfidf_vectorizer.transform([data]) # converting to vectors
    data = model_randomforest_best.predict(data.toarray()) # prediction
    data = label_encoder.inverse_transform(data) #inverse encoding of prediction
    return data[0]

In [63]:
test_model_rf('for every action there is equal and opposite reaction')

'Chemistry'

In [64]:
test_model_rf('protien is the main constituent of DNA')

'Chemistry'

In [65]:
test_model_rf('pH of any solution less than 7, is acidic')

'Chemistry'

In [66]:
test_model_svc('for every action there is equal and opposite reaction')

'Chemistry'

In [68]:
test_model_rf('force of gravitation is the highest')

'Physics'

In [69]:
test_model_rf('water contains oxygen and hydrogen in 2:1')

'Chemistry'

In [71]:
test_model_svc('hello cells this is ameoba')

'Biology'

### Hyperparameter tunning(SVC)

In [73]:
param_grid = {'C': [0.1, 1, 10, 100],
              'kernel': ['linear', 'rbf', 'poly'],
              'gamma': [0.1, 0.01, 0.001],
              'degree': [2, 3, 4]}

In [74]:
%%time
grid_search = GridSearchCV(model_scv, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(vector_tfidf.toarray(), df.Topic_Code)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


KeyboardInterrupt: 

In [75]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
print("Best Parameters (SVC):", best_params)
print("Best Score (SCV):", best_score)

In [ ]:
test_score = grid_search.score(tfidf_vectorizer.transform(df_test.Cleaned_Comments).toarray(), df_test.Topic_Code)
print("Test Score:", test_score)